# CS395 - Assignment 6
### Multi-Step LSTMs

Date: March 3, 2019  
By: Joshua Eli Swick

## Data Preparation

In [3]:
# Imports

# multi-step data preparation
from numpy import array 

# univariate multi-step vector-output stacked lstm example 
from keras.models import Sequential 
from keras.layers import LSTM 
from keras.layers import Dense 

# univariate multi-step encoder-decoder lstm example
from keras.layers import RepeatVector 
from keras.layers import TimeDistributed 

# bidirectional lstm
from keras.layers import Bidirectional

# multivariate multi-step data preparation
from numpy import hstack

Using TensorFlow backend.


In [40]:
# split a univariate sequence into samples 
def split_sequence(sequence, n_steps_in, n_steps_out):
    X, y = list(), list() 
    for i in range(len(sequence)): 
        # find the end of this pattern 
        end_ix = i + n_steps_in 
        out_end_ix = end_ix + n_steps_out 
        # check if we are beyond the sequence 
        if out_end_ix > len(sequence): 
              break 
        # gather input and output parts of the pattern 
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix] 
        X.append(seq_x) 
        y.append(seq_y) 
    return array(X), array(y)

### 1. 
Run given code and show results.

In [41]:
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps_in, n_steps_out = 3, 2
# split into samples
X, y = split_sequence(raw_seq, n_steps_in, n_steps_out)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i]) 

[10 20 30] [40 50]
[20 30 40] [50 60]
[30 40 50] [60 70]
[40 50 60] [70 80]
[50 60 70] [80 90]


## Vector Output Model

### 2. 
Run the given code 3 times. Show input once and all outputs.

In [13]:
# reshape from [samples, timesteps] into [samples, timesteps, features] 
n_features = 1 
X = X.reshape((X.shape[0], X.shape[1], n_features))

for run in range(3):
    # define model 
    model = Sequential() 
    model.add(LSTM(100, activation='relu', return_sequences=True, 
                   input_shape=(n_steps_in, n_features))) 
    model.add(LSTM(100, activation='relu')) 
    model.add(Dense(n_steps_out)) 
    model.compile(optimizer='adam', loss='mse') 

    # fit model 
    model.fit(X, y, epochs=50, verbose=0) 

    # demonstrate prediction 
    x_input = array([70, 80, 90]) 
    x_input = x_input.reshape((1, n_steps_in, n_features)) 
    yhat = model.predict(x_input, verbose=0) 

    print(f"Vector Output LSTM, run {run+1}: {yhat}") 

print(f"\nx_input: \n{x_input}")

Vector Output LSTM, run 1: [[102.79567  116.121346]]
Vector Output LSTM, run 2: [[101.06105 115.52495]]
Vector Output LSTM, run 3: [[100.88695 129.13826]]

x_input: 
[[[70]
  [80]
  [90]]]


### 3. 
Change the activation function from 'relu' to 2 other activation functions. Run each function 3 times. Show all outputs.

In [36]:
for run in range(3):
    # define model 
    model = Sequential() 
    model.add(LSTM(100, activation='selu', return_sequences=True,  # selu instead of relu
                   input_shape=(n_steps_in, n_features))) 
    model.add(LSTM(100, activation='selu')) 
    model.add(Dense(n_steps_out)) 
    model.compile(optimizer='adam', loss='mse') 

    # fit model 
    model.fit(X, y, epochs=50, verbose=0) 

    # demonstrate prediction 
    x_input = array([70, 80, 90]) 
    x_input = x_input.reshape((1, n_steps_in, n_features)) 
    yhat = model.predict(x_input, verbose=0) 

    print(f"Vector Output LSTM, selu activation function, run {run+1}: {yhat}") 

Vector Output LSTM, selu activation function, run 1: [[105.630646 118.06458 ]]
Vector Output LSTM, selu activation function, run 2: [[106.83967 121.45907]]
Vector Output LSTM, selu activation function, run 3: [[107.29903 123.11093]]


In [37]:
for run in range(3):
    # define model 
    model = Sequential() 
    model.add(LSTM(100, activation='elu', return_sequences=True,  # elu instead of relu
                   input_shape=(n_steps_in, n_features))) 
    model.add(LSTM(100, activation='elu')) 
    model.add(Dense(n_steps_out)) 
    model.compile(optimizer='adam', loss='mse') 

    # fit model 
    model.fit(X, y, epochs=50, verbose=0) 

    # demonstrate prediction 
    x_input = array([70, 80, 90]) 
    x_input = x_input.reshape((1, n_steps_in, n_features)) 
    yhat = model.predict(x_input, verbose=0) 

    print(f"Vector Output LSTM, elu activation function, run {run+1}: {yhat}") 

Vector Output LSTM, elu activation function, run 1: [[109.80136 122.74441]]
Vector Output LSTM, elu activation function, run 2: [[111.408516 124.25589 ]]
Vector Output LSTM, elu activation function, run 3: [[104.200806 120.476204]]


## Encoder-Decoder Model

### 4.
Run the given code 3 times. Show input once and all outputs.

In [14]:
X = X.reshape((X.shape[0], X.shape[1], n_features))
y = y.reshape((y.shape[0], y.shape[1], n_features))

for run in range(3):
    # define model 
    model = Sequential() 
    model.add(LSTM(100, activation='relu', input_shape=(n_steps_in, n_features))) 
    model.add(RepeatVector(n_steps_out)) 
    model.add(LSTM(100, activation='relu', return_sequences=True)) 
    model.add(TimeDistributed(Dense(1))) 
    model.compile(optimizer='adam', loss='mse') 
    # fit model 
    model.fit(X, y, epochs=100, verbose=0) 
    # demonstrate prediction 
    x_input = array([70, 80, 90]) 
    x_input = x_input.reshape((1, n_steps_in, n_features)) 
    yhat = model.predict(x_input, verbose=0) 
    print(f"Encoder-Decoder LSTM, run {run+1}: \n{yhat}") 

print(f"\nx_input: \n{x_input}")

Encoder-Decoder LSTM, run 1: 
[[[106.222305]
  [118.724464]]]
Encoder-Decoder LSTM, run 2: 
[[[107.75766]
  [125.63242]]]
Encoder-Decoder LSTM, run 3: 
[[[104.9773 ]
  [117.74535]]]

x_input: 
[[[70]
  [80]
  [90]]]


### 5.
Change the activation function from 'relu' to 2 other activation functions. Run each function 3 times. Show all outputs.

In [40]:
for run in range(3):
    # define model 
    model = Sequential() 
    model.add(LSTM(100, activation='elu', input_shape=(n_steps_in, n_features)))  # elu activation function 
    model.add(RepeatVector(n_steps_out)) 
    model.add(LSTM(100, activation='elu', return_sequences=True)) 
    model.add(TimeDistributed(Dense(1))) 
    model.compile(optimizer='adam', loss='mse') 
    # fit model 
    model.fit(X, y, epochs=100, verbose=0) 
    # demonstrate prediction 
    x_input = array([70, 80, 90]) 
    x_input = x_input.reshape((1, n_steps_in, n_features)) 
    yhat = model.predict(x_input, verbose=0) 
    print(f"Encoder-Decoder LSTM, elu activation function, run {run+1}: \n{yhat}") 

Encoder-Decoder LSTM, elu activation function, run 1: 
[[[ 97.71329 ]
  [111.263145]]]
Encoder-Decoder LSTM, elu activation function, run 2: 
[[[ 98.90968]
  [111.2552 ]]]
Encoder-Decoder LSTM, elu activation function, run 3: 
[[[104.62835]
  [113.66432]]]


In [41]:
for run in range(3):
    # define model 
    model = Sequential() 
    model.add(LSTM(100, activation='selu', input_shape=(n_steps_in, n_features)))  # selu activation function 
    model.add(RepeatVector(n_steps_out)) 
    model.add(LSTM(100, activation='selu', return_sequences=True)) 
    model.add(TimeDistributed(Dense(1))) 
    model.compile(optimizer='adam', loss='mse') 
    # fit model 
    model.fit(X, y, epochs=100, verbose=0) 
    # demonstrate prediction 
    x_input = array([70, 80, 90]) 
    x_input = x_input.reshape((1, n_steps_in, n_features)) 
    yhat = model.predict(x_input, verbose=0) 
    print(f"Encoder-Decoder LSTM, selu activation function, run {run+1}: \n{yhat}") 

Encoder-Decoder LSTM, selu activation function, run 1: 
[[[102.055504]
  [112.62822 ]]]
Encoder-Decoder LSTM, selu activation function, run 2: 
[[[ 98.53076 ]
  [113.305176]]]
Encoder-Decoder LSTM, selu activation function, run 3: 
[[[102.606995]
  [116.574745]]]


### 6.
Change the activation function back to 'relu'. Change this model to two of the other univariate LSTM models (i.e. Stacked, Bidirectional, CNN, or Conv LSTM) to create an Encoder-Decoder Model <x> LSTM, where <x> is the other univariate function model. Run 3 times, show all outputs.

In [46]:
for run in range(3):
    # define model 
    model = Sequential() 
    model.add(Bidirectional(LSTM(100, activation='relu'), input_shape=(n_steps_in, n_features)))
    model.add(RepeatVector(n_steps_out)) 
    model.add(LSTM(100, activation='relu', return_sequences=True)) 
    model.add(TimeDistributed(Dense(1))) 
    model.compile(optimizer='adam', loss='mse') 
    # fit model 
    model.fit(X, y, epochs=100, verbose=0) 
    # demonstrate prediction 
    x_input = array([70, 80, 90]) 
    x_input = x_input.reshape((1, n_steps_in, n_features)) 
    yhat = model.predict(x_input, verbose=0) 
    print(f"Encoder-Decoder Bidirectional LSTM, relu activation function, run {run+1}: \n{yhat}") 

Encoder-Decoder Bidirectional LSTM, relu activation function, run 1: 
[[[101.18164 ]
  [113.611176]]]
Encoder-Decoder Bidirectional LSTM, relu activation function, run 2: 
[[[102.268974]
  [117.79098 ]]]
Encoder-Decoder Bidirectional LSTM, relu activation function, run 3: 
[[[102.80594]
  [116.99763]]]


In [27]:
for run in range(3):
    # define model 
    model = Sequential() 
    model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps_in, n_features)))
    model.add(LSTM(100, activation='relu'))
    model.add(RepeatVector(n_steps_out)) 
    model.add(LSTM(100, activation='relu', return_sequences=True)) 
    model.add(TimeDistributed(Dense(1))) 
    model.compile(optimizer='adam', loss='mse') 
    # fit model 
    model.fit(X, y, epochs=100, verbose=0) 
    # demonstrate prediction 
    x_input = array([70, 80, 90]) 
    x_input = x_input.reshape((1, n_steps_in, n_features)) 
    yhat = model.predict(x_input, verbose=0) 
    print(f"Encoder-Decoder Bidirectional LSTM, relu activation function, run {run+1}: \n{yhat}") 

Encoder-Decoder Bidirectional LSTM, relu activation function, run 1: 
[[[105.266975]
  [121.959465]]]
Encoder-Decoder Bidirectional LSTM, relu activation function, run 2: 
[[[103.31849 ]
  [122.195526]]]
Encoder-Decoder Bidirectional LSTM, relu activation function, run 3: 
[[[100.53522 ]
  [113.311424]]]


## Multiple Input Multi-Step Output

### 7. 
Run the given code, show all outputs.

In [42]:
# define input sequence 
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90]) 
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95]) 
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))]) 

# convert to [rows, columns] structure 
in_seq1 = in_seq1.reshape((len(in_seq1), 1)) 
in_seq2 = in_seq2.reshape((len(in_seq2), 1)) 
out_seq = out_seq.reshape((len(out_seq), 1)) 

# horizontally stack columns 
dataset = hstack((in_seq1, in_seq2, out_seq)) 

# choose a number of time steps 
n_steps_in, n_steps_out = 3, 2 

# covert into input/output 
X, y = split_sequence(dataset, n_steps_in, n_steps_out) 
print(X.shape, y.shape) 

# summarize the data 
for i in range(len(X)):
    print(X[i], y[i]) 

(5, 3, 3) (5, 2, 3)
[[10 15 25]
 [20 25 45]
 [30 35 65]] [[ 40  45  85]
 [ 50  55 105]]
[[20 25 45]
 [30 35 65]
 [40 45 85]] [[ 50  55 105]
 [ 60  65 125]]
[[ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]] [[ 60  65 125]
 [ 70  75 145]]
[[ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]] [[ 70  75 145]
 [ 80  85 165]]
[[ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]] [[ 80  85 165]
 [ 90  95 185]]


### 8.
Run the given code 3 times. Show input once and all outputs.

In [49]:
# multivariate multi-step stacked lstm example
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out-1
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
# choose a number of time steps
n_steps_in, n_steps_out = 3, 2
# covert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
for run in range(3):
    # define model
    model = Sequential()
    model.add(LSTM(100, activation='relu', return_sequences=True,
    input_shape=(n_steps_in, n_features)))
    model.add(LSTM(100, activation='relu'))
    model.add(Dense(n_steps_out))
    model.compile(optimizer='adam', loss='mse')
    # fit model
    model.fit(X, y, epochs=200, verbose=0)
    # demonstrate prediction
    x_input = array([[70, 75], [80, 85], [90, 95]])
    x_input = x_input.reshape((1, n_steps_in, n_features))
    yhat = model.predict(x_input, verbose=0)
    print(f"Multivariate multi-step stacked LSTM, run {run+1}: {yhat}")
    
print(f"\nInput: \n{x_input}")

Multivariate multi-step stacked LSTM, run 1: [[187.53465 209.63057]]
Multivariate multi-step stacked LSTM, run 2: [[187.53015 208.98615]]
Multivariate multi-step stacked LSTM, run 3: [[186.30579 207.78137]]

Input: 
[[[70 75]
  [80 85]
  [90 95]]]


### 9.
Change the activation function from 'relu' to 2 other activation functions. Run each function 3 times. Show all outputs.

In [48]:
for run in range(3):
    # define model
    model = Sequential()
    model.add(LSTM(100, activation='elu', return_sequences=True,
    input_shape=(n_steps_in, n_features)))
    model.add(LSTM(100, activation='elu'))
    model.add(Dense(n_steps_out))
    model.compile(optimizer='adam', loss='mse')
    # fit model
    model.fit(X, y, epochs=200, verbose=0)
    # demonstrate prediction
    x_input = array([[70, 75], [80, 85], [90, 95]])
    x_input = x_input.reshape((1, n_steps_in, n_features))
    yhat = model.predict(x_input, verbose=0)
    print(f"Multivariate multi-step stacked LSTM, elu activation function, run {run+1}: {yhat}")

Multivariate multi-step stacked LSTM, elu activation function, run 1: [[187.28606 209.56604]]
Multivariate multi-step stacked LSTM, elu activation function, run 2: [[185.96056 208.4713 ]]
Multivariate multi-step stacked LSTM, elu activation function, run 3: [[186.3392  208.25137]]


In [50]:
for run in range(3):
    # define model
    model = Sequential()
    model.add(LSTM(100, activation='selu', return_sequences=True,
    input_shape=(n_steps_in, n_features)))
    model.add(LSTM(100, activation='selu'))
    model.add(Dense(n_steps_out))
    model.compile(optimizer='adam', loss='mse')
    # fit model
    model.fit(X, y, epochs=200, verbose=0)
    # demonstrate prediction
    x_input = array([[70, 75], [80, 85], [90, 95]])
    x_input = x_input.reshape((1, n_steps_in, n_features))
    yhat = model.predict(x_input, verbose=0)
    print(f"Multivariate multi-step stacked LSTM, selu activation function, run {run+1}: {yhat}")

Multivariate multi-step stacked LSTM, selu activation function, run 1: [[184.79706 206.01259]]
Multivariate multi-step stacked LSTM, selu activation function, run 2: [[184.9093  206.20804]]
Multivariate multi-step stacked LSTM, selu activation function, run 3: [[186.09892 206.44264]]


## Multiple Parallel Input and Multi-Step Output

### 10.
Run given code and show all outputs.

In [52]:
# multivariate multi-step data preparation
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
        return array(X), array(y)
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
# choose a number of time steps
n_steps_in, n_steps_out = 3, 2
# covert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)
print(X.shape, y.shape)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

(1, 3, 3) (1, 2, 3)
[[10 15 25]
 [20 25 45]
 [30 35 65]] [[ 40  45  85]
 [ 50  55 105]]


### 11.
Run the given code 3 times. Show input once and all outputs.

In [6]:
# multivariate multi-step encoder-decoder lstm example
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
# choose a number of time steps
n_steps_in, n_steps_out = 3, 2
# covert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
for run in range(3):
    # define model
    model = Sequential()
    model.add(LSTM(200, activation='relu', input_shape=(n_steps_in, n_features)))
    model.add(RepeatVector(n_steps_out))
    model.add(LSTM(200, activation='relu', return_sequences=True))
    model.add(TimeDistributed(Dense(n_features)))
    model.compile(optimizer='adam', loss='mse')
    # fit model
    model.fit(X, y, epochs=300, verbose=0)
    # demonstrate prediction
    x_input = array([[60, 65, 125], [70, 75, 145], [80, 85, 165]])
    x_input = x_input.reshape((1, n_steps_in, n_features))
    yhat = model.predict(x_input, verbose=0)
    print(f"Multivariate multi-step encoder-decoder LSTM, run {run+1}: \n{yhat}")

print(f"\nInput: \n{x_input}")

Multivariate multi-step encoder-decoder LSTM, run 1: 
[[[ 91.814384  97.35714  189.79308 ]
  [102.57893  108.27749  212.25337 ]]]
Multivariate multi-step encoder-decoder LSTM, run 2: 
[[[ 90.856224  96.021126 187.33286 ]
  [101.30607  106.89735  209.14952 ]]]
Multivariate multi-step encoder-decoder LSTM, run 3: 
[[[ 91.2538   97.36612 188.6374 ]
  [103.67517 108.71683 212.24504]]]

Input: 
[[[ 60  65 125]
  [ 70  75 145]
  [ 80  85 165]]]


### 12.
Change the activation function from 'relu' to 2 other activation functions. Run each function 3 times. Show all outputs.

In [7]:
for run in range(3):
    # define model
    model = Sequential()
    model.add(LSTM(200, activation='selu', input_shape=(n_steps_in, n_features)))
    model.add(RepeatVector(n_steps_out))
    model.add(LSTM(200, activation='relu', return_sequences=True))
    model.add(TimeDistributed(Dense(n_features)))
    model.compile(optimizer='adam', loss='mse')
    # fit model
    model.fit(X, y, epochs=300, verbose=0)
    # demonstrate prediction
    x_input = array([[60, 65, 125], [70, 75, 145], [80, 85, 165]])
    x_input = x_input.reshape((1, n_steps_in, n_features))
    yhat = model.predict(x_input, verbose=0)
    print(f"Multivariate multi-step encoder-decoder LSTM, selu activation function, run {run+1}: \n{yhat}")

Multivariate multi-step encoder-decoder LSTM, selu activation function, run 1: 
[[[ 90.20457   95.72145  186.1131  ]
  [101.35735  106.545975 208.0106  ]]]
Multivariate multi-step encoder-decoder LSTM, selu activation function, run 2: 
[[[ 90.3685   95.83061 186.33356]
  [101.23498 106.79179 208.53351]]]
Multivariate multi-step encoder-decoder LSTM, selu activation function, run 3: 
[[[ 90.61505  96.22584 186.97276]
  [101.72075 107.37853 209.07985]]]


In [61]:
for run in range(3):
    # define model
    model = Sequential()
    model.add(LSTM(200, activation='selu', input_shape=(n_steps_in, n_features)))
    model.add(RepeatVector(n_steps_out))
    model.add(LSTM(200, activation='relu', return_sequences=True))
    model.add(TimeDistributed(Dense(n_features)))
    model.compile(optimizer='adam', loss='mse')
    # fit model
    model.fit(X, y, epochs=300, verbose=0)
    # demonstrate prediction
    x_input = array([[60, 65, 125], [70, 75, 145], [80, 85, 165]])
    x_input = x_input.reshape((1, n_steps_in, n_features))
    yhat = model.predict(x_input, verbose=0)
    print(f"Multivariate multi-step encoder-decoder LSTM, elu activation function, run {run+1}: \n{yhat}")

Multivariate multi-step encoder-decoder LSTM, elu activation function, run 1: 
[[[ 90.93138  95.72269 185.8148 ]
  [101.75057 106.31448 207.9682 ]]]
Multivariate multi-step encoder-decoder LSTM, elu activation function, run 2: 
[[[ 90.77438   95.68702  186.27686 ]
  [101.699715 106.48441  207.80688 ]]]
Multivariate multi-step encoder-decoder LSTM, elu activation function, run 3: 
[[[ 90.35345   95.95481  186.48857 ]
  [101.557785 107.302605 208.80862 ]]]


### 13.
Provide a single table that illustrates the different LSTM models, the activation functions used, and the average of the 3 results (predictions) to 3 decimal places. What are the general findings?

| LSTM Model                              | Activation Function | Avg. of 3 Results | General Findings |  
| --------------------------------------- | ------------------- | ----------------- | ---------------- |
| Vector Output                           | Relu                | [101.581 120.261] | 4th              |
| Vector Output                           | Selu                | [106.590 120.878] | 7th              |
| Vector Output                           | Elu                 | [108.470 122.492] | 8th              |
| Encoder-Decoder                         | Relu                | [106.319 120.700] | 6th              |
| Encoder-Decoder                         | Elu                 | [100.417 122.061] | 5th              |
| Encoder-Decoder                         | Selu                | [101.067 114.169] | 1st              |
| Bidirectional                           | Relu                | [102.086 116.133] | 2nd              |
| Stacked                                 | Relu                | [103.040 119.155] | 3rd              |
|                                                                                                      |  
| Multivariate Multi-Step Stacked         | Relu                | [187.124 208.799] | 3rd              |
| Multivariate Multi-Step Stacked         | Elu                 | [186.529 208.763] | 2nd              |
| Multivariate Multi-Step Stacked         | Selu                | [185.268 206.221] | 1st              |
|                                                                                                      |  
| Multivariate Multi-Step Encoder-Decoder | Relu                | [090.873 096.239 187.261] | 3rd              |
| -                                       | -                   | [101.814 107.101 209.545] | -                |
| Multivariate Multi-Step Encoder-Decoder | Elu                 | [090.396 095.926 186.473] | 2nd              |
| -                                       | -                   | [101.438 106.906 208.542] | -                |
| Multivariate Multi-Step Encoder-Decoder | Selu                | [090.686 095.788 186.194] | 1st              |
| -                                       | -                   | [101.670 106.700 208.195] | -                |

Overall the results for each LSTM model were quite accurate. The Selu Activation Function seemed to produce the most accurate predictions, while Relu was the least accurate for Multivariate Multi-Step models and Elu was the least accurate Activation Function for the Univariate models.